##### Here I am implementing SVM on a real like dataset like:cancer dataset.I will be implementing the loss functions also and checking on the test dataset. To optimize the loss function I will be using SGD.

In [1]:
# imports 
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [2]:
c=10000 # C parameter: regularization parameter
lr=0.000001 # learning rate to optimize weight and bias during SGD

In [3]:
# defining cost function
def cost_fun(w,X,y):
    # calculating the 2 losses and then adding them.
    n=X.shape[0]
    dist=1-y*(np.dot(X,w))
    dist[dist<0]=0 # eq to max(0,dist)
    hinge_loss=c*(np.sum(dist)/n)
    # Calculate total loss function
    cost=1/2*np.dot(w,w)+hinge_loss
    return cost

In [4]:
# Calculating cost gradients
def calculate_gradient(w,X_batch,y_batch):
    # if only one example is passed like in SGD
    if type(y_batch)==np.float64:
        y_batch=np.array([y_batch])
        X_batch=np.array([X_batch])

    distance=1-(y_batch*np.dot(X_batch,w))
    dw=np.zeros(len(w))

    for i,d in enumerate(distance):
        if max(0,d)==0:
            di=w
        else:
            di=w-(c*y_batch[i]*X_batch[i])
        dw+=di

    dw=dw/len(y_batch) 
    return dw

In [5]:
# Defining SGD with stoppage criteria to check if cost has decreased much or not.
from sklearn.utils import shuffle
def sgd(feature,op):
    max_epochs=4000 # can be some other number also
    weights=np.zeros(feature.shape[1])
    nth=0 # I will check cost at some regular interval so using this.
    prev_cost=float("inf")
    cost_threshold=0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(feature, op)
        for ind, x in enumerate(X):
            ascent = calculate_gradient(weights, x, Y[ind])
            weights = weights-(lr*ascent)

        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = cost_fun(weights, feature,op)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights

In [6]:
df=pd.read_csv('data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [7]:
 # drop last column (extra column added by pd) 
 # and unnecessary first column (id)
df.drop(df.columns[[-1, 0]], axis=1, inplace=True)

In [8]:
# convert categorical labels to numbers
diag_map = {'M': 1.0, 'B': -1.0}
df['diagnosis'] = df['diagnosis'].map(diag_map)


In [14]:
def predict():
    # put features & outputs in different data frames
    Y = df.loc[:,'diagnosis']
    X = df.iloc[:, 1:]
    # normalize data for better convergence and to prevent overflow
    X_normalized = MinMaxScaler().fit_transform(X.values)
    X = pd.DataFrame(X_normalized)

    # insert 1 in every row for intercept b
    X.insert(loc=len(X.columns), column='intercept', value=1)

    # split data into train and test set
    X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)

    # train the model
    W = sgd(X_train.to_numpy(), y_train.to_numpy())
    print("The updated weights after SGD are: {}".format(W))

    # testing the model
    y_train_predicted = np.array([])
    for i in range(X_train.shape[0]):
        yp = np.sign(np.dot(X_train.to_numpy()[i], W))
        y_train_predicted = np.append(y_train_predicted, yp)

    y_test_predicted = np.array([])
    for i in range(X_test.shape[0]):
        yp = np.sign(np.dot(X_test.to_numpy()[i], W))
        y_test_predicted = np.append(y_test_predicted, yp)

    print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_test_predicted)))
    print("recall on test dataset: {}".format(recall_score(y_test, y_test_predicted)))
    print("precision on test dataset: {}".format(precision_score(y_test, y_test_predicted)))

In [15]:
predict()

Epoch is: 1 and Cost is: 5428.916904545349
Epoch is: 2 and Cost is: 3420.1010392906123
Epoch is: 4 and Cost is: 2397.7127694008773
Epoch is: 8 and Cost is: 1877.2392237464794
Epoch is: 16 and Cost is: 1513.9145942126463
Epoch is: 32 and Cost is: 1208.131361998415
Epoch is: 64 and Cost is: 975.0913500879705
Epoch is: 128 and Cost is: 805.6662791026421
Epoch is: 256 and Cost is: 707.1477218098838
Epoch is: 512 and Cost is: 645.1136078741729
Epoch is: 1024 and Cost is: 625.5211396380479
Epoch is: 2048 and Cost is: 610.4548592777709
Epoch is: 3999 and Cost is: 605.0125659067194
The updated weights after SGD are: [ 1.27151912e+00  9.07456010e-01  1.08573541e+00  2.15661231e+00
 -1.22522368e+00 -3.12890395e+00  3.32354793e+00  6.69115931e+00
 -4.87921474e-01  7.09733899e-03  5.59712310e+00 -1.82467620e+00
  3.27436737e+00  3.69309315e+00  1.66682983e+00 -2.41180610e+00
 -1.65792823e+00  7.08037864e-01 -1.90594904e+00 -1.76365519e+00
  2.72608366e+00  5.25914070e+00  1.14693655e+00  3.0818334